In [2]:
import os

sentiment_directory = '../clustered_data/sentiment_scores/'
sentiment_files = [f for f in os.listdir(sentiment_directory) if f.endswith('gemini.csv')]
sentiment_files

['CNN-Judge orders Trump to pay $355 million in civil fraud trial_gemini.csv',
 'Fox News-Trump ordered to pay $364M, found liable in civil fraud trial_gemini.csv']

In [52]:
import os

import google.generativeai as genai
import warnings
warnings.filterwarnings("ignore")

GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

from google.generativeai.types import HarmCategory, HarmBlockThreshold

def target_detection(text, video_title):
    model = genai.GenerativeModel('gemini-pro')
    message = "The following text is a comment on the video titled: " + video_title + ". Identify the political target of the comment within the given political context and return ONLY the target: " + text
    try:
        response = model.generate_content(message, safety_settings={
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
            }
        )
        target = response.text
        return target
    except AttributeError as e:
        model = genai.GenerativeModel('gemini-pro')
        return None, None
    except Exception as e:
        model = genai.GenerativeModel('gemini-pro')
        return None, None
    
def stance_detection(text, video_title, target):
    model = genai.GenerativeModel('gemini-pro')
    message = "The following text is a comment on the video titled: " + video_title + ". Identify the stance of the comment towards the target within the given political context and ONLY return the stance (AGAINST, PRO): " + text
    try:
        response = model.generate_content(message, safety_settings={
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
            }
        )
        stance = response.text
        return stance
    except AttributeError as e:
        model = genai.GenerativeModel('gemini-pro')
        return None, None
    except Exception as e:
        model = genai.GenerativeModel('gemini-pro')
        return None, None
    
def target_stance_detection(text, video_title):
    target = target_detection(text, video_title)
    stance = stance_detection(text, video_title, target)
    return target, stance
    
target_stance_detection("This verdict VIOLATES THE 8th amendment of the constitution of the United States. Shame on any judge that would knowingly violate their oath of office to support the constitution.This judge should be disbarred \nIf they do this to him they can do it to you", "Judge orders Trump to pay $355 million in civil fraud trial")

('Judge', 'AGAINST')

In [53]:
import pandas as pd
import os

def save_stance(sentiment_file_path, file_name, output_path):
    with open(sentiment_file_path + file_name, 'r') as f:
        df = pd.read_csv(f)
        video_title = file_name.split('.')[0].removesuffix('_gemini')
        df['target'] = None
        df['stance'] = None
        # for i, row in df.iterrows():
        #     target = target_detection(row['comment'], video_title)
        #     stance = stance_detection(row['comment'], video_title, target)
        #     df.at[i, 'target'] = target
        #     df.at[i, 'stance'] = stance
        # Choose random sample of 10 comments to test
        sample = df.sample(10)
        sample.drop(columns=['sentiment_textblob'], inplace=True)
        for i, row in sample.iterrows():
            print(sample)
            target, stance = target_stance_detection(row['comment'], video_title)
            sample.at[i, 'target'] = target
            sample.at[i, 'stance'] = stance
        sample.to_csv(output_path, index=False)

In [54]:
sentiment_directory = '../clustered_data/sentiment_scores/'
output_directory = '../test_output/stance_detection/'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
log_time = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
sentiment_files = [f for f in os.listdir(sentiment_directory) if f.endswith('_gemini.csv')]
for f in sentiment_files:
    save_stance(sentiment_directory, f, output_directory + f.split('.')[0].removesuffix('_gemini') + f'_{log_time}.csv')

                                                comment  sentiment_gemini  \
2066  trump will begging the american people hard bi...              -1.0   
1274  wasnt enough hope the trumps are banned from d...              -0.6   
925   yet another miscarriage of justice but trump w...              -0.5   
141         give elie jean hers first she needs a break               0.7   
182   appeal pending its not over till its over not ...               0.6   
1444  ha ha trump wont pay shit even hitler wouldnt ...              -0.7   
937   bad decisions have bad consequences reap it trump              -0.5   
964   this nonsense case will get reversed and im no...              -0.5   
1016  there will be a price to pay for what these co...              -0.5   
787   republicans are okay with paying his bills for...              -0.5   

      sentiment_vader target stance  
2066          -0.1027   None   None  
1274          -0.8077   None   None  
925            0.8883   None   None  
